# データセットの収集と転移学習


In [ ]:
import Jetson.GPIO as GPIO

BOARD_NAME=GPIO.gpio_pin_data.get_data()[0]
if BOARD_NAME == "JETSON_NX":
    print("Jetson Xavier NXを認識")
    I2C_BUSNUM = 8
    MODE = 3
elif BOARD_NAME == "JETSON_XAVIER":
    print("Jetson AGX Xavierを認識")
    I2C_BUSNUM = 8
    MODE = 2
elif BOARD_NAME == "JETSON_NANO":
    print("Jetson NANOを認識")
    I2C_BUSNUM = 1
    MODE = 0

In [ ]:
!echo "jetson" | sudo -S nvpmodel -m $MODE

In [ ]:
!echo "jetson" | sudo -S nvpmodel -q

In [ ]:
!echo "jetson" | sudo -S jetson_clocks

### ログ用のWidgewt

In [ ]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label

l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

### カメラの初期化

In [ ]:
from jetcam.csi_camera import CSICamera
# from jetcam.usb_camera import USBCamera

camera = CSICamera(width=224, height=224)
# camera = USBCamera(width=224, height=224)

camera.running = True

### データセットのタスクの定義

In [ ]:
import torchvision.transforms as transforms
from xy_dataset import XYDataset
import os
import subprocess

TASKS = ['interactive', 'train']

CATEGORIES = ['xy', 'speed']

DATASETS = ['A', 'B', 'C', 'D']
DATASETS_TRAIN = ['RE_A', 'RE_B', 'RE_C', 'RE_D']

for name in DATASETS:
    current_path = os.getcwd()
    path = current_path + "/" + TASKS[0] + "/" + name + "/"
    if not os.path.exists(path):
        subprocess.call(['mkdir', '-p', path])            

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = XYDataset(TASKS[0] + '/' + name, CATEGORIES, TRANSFORMS, random_hflip=True)
    print(TASKS[0] + '/' + name)
for name in DATASETS_TRAIN:
    datasets[name] = XYDataset(TASKS[1] + '/' + name, CATEGORIES, TRANSFORMS, random_hflip=True)
    print(TASKS[1] + '/' + name)

### データ収集

In [ ]:
import cv2
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget
import os

# initialize active dataset
dataset = datasets[DATASETS[0]]
#print(DATASETS[0])
# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

# create widgets
task_widget = ipywidgets.Dropdown(options=TASKS, description='task')
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_xy_widget = ipywidgets.IntText(description='xy count')
count_speed_widget = ipywidgets.IntText(description='speed count')

# manually update counts at initialization
count_xy_widget.value = dataset.get_count(CATEGORIES[0])
count_speed_widget.value = dataset.get_count(CATEGORIES[1])

# sets the active dataset
def set_dataset(change):
    global dataset, dataset_widget, task_widget
    try:
        dataset = XYDataset(task_widget.value + '/' + dataset_widget.value, CATEGORIES, TRANSFORMS, random_hflip=True)
        #dataset = datasets[change['new']]
        count_xy_widget.value = dataset.get_count(CATEGORIES[0])
        count_speed_widget.value = dataset.get_count(CATEGORIES[1])
    except:
        write_log("データセットが存在していません。")
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_xy_widget.value = dataset.get_count(CATEGORIES[0])
    count_speed_widget.value = dataset.get_count(CATEGORIES[1])
category_widget.observe(update_counts, names='value')


def change_task(change):
    global dataset_widget, task_widget
    try:
        current_path = os.getcwd()
        path = current_path + "/" + task_widget.value + "/"
        files = os.listdir(path)
        dirs = [f for f in files if os.path.isdir(os.path.join(path, f))]
        dirs = sorted(dirs)
        dataset_widget.options = dirs
    except:
        write_log(path + "が存在していません。")
        dataset_widget.options = []
task_widget.observe(change_task, names='value')
change_task(TASKS[0])

def save_snapshot(_, content, msg):
    global process_widget
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # save to disk
        dataset.save_entry(category_widget.value, camera.value, x, y)
        
        # display saved snapshot
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_xy_widget.value = dataset.get_count("xy")
        write_log("count:"+str(count_xy_widget.value))
        count_speed_widget.value = dataset.get_count(CATEGORIES[1])
        write_log("xy (" + str(x) + "," + str(y) + ")のポイントでデータを登録しました。")
        
camera_widget.on_msg(save_snapshot)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]),
    ipywidgets.HBox([ipywidgets.VBox([dataset_widget,count_xy_widget,count_speed_widget]),task_widget]),
])

display(data_collection_widget)

### 転移学習用の学習済みモデルを読み込み

In [ ]:
import subprocess
import datetime
import os
import glob

model_load_widget = ipywidgets.Dropdown(options=[],description='読込モデル')
model_load_time_widget = ipywidgets.Label(description='作成日時：')
model_save_name_widget = ipywidgets.Text(description='保存モデル名',value="model.pth")

def model_list(change):
    global model_load_widget
    try:
        files = glob.glob('./model/*.pth', recursive=True)
        files.insert(0,"[new]")
        model_load_widget.options = files
        model_load_time_widget.value = '作成日時：'
    except:
        model_load_widget.options = []
model_list("list")

def change_file(change):
    global model_load_widget
    try:
        file = model_load_widget.value
        ts = os.path.getctime(file)
        d = datetime.datetime.fromtimestamp(ts)
        s = d.strftime('%Y-%m-%d %H:%M:%S')
        model_load_time_widget.value = f'作成日時：{s}'
    except:
        model_load_time_widget.value = '作成日時：'
model_load_widget.observe(change_file, names='value')

In [ ]:
import torch
import torchvision
import os
import subprocess

device = torch.device('cuda')
output_dim = 2 * len(dataset.categories)  # x, y coordinate for each category

# ALEXNET
# model = torchvision.models.alexnet(pretrained=True)
# model.classifier[-1] = torch.nn.Linear(4096, output_dim)

# SQUEEZENET 
# model = torchvision.models.squeezenet1_1(pretrained=True)
# model.classifier[1] = torch.nn.Conv2d(512, output_dim, kernel_size=1)
# model.num_classes = len(dataset.categories)

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)

# RESNET 34
# model = torchvision.models.resnet34(pretrained=True)
# model.fc = torch.nn.Linear(512, output_dim)

# DENSENET 121
# model = torchvision.models.densenet121(pretrained=True)
# model.classifier = torch.nn.Linear(model.num_features, output_dim)

model = model.to(device)

model_save_button = ipywidgets.Button(description='save model')
model_load_button = ipywidgets.Button(description='load model')

def load_model(c):
    model.load_state_dict(torch.load(model_load_widget.value))
    write_log(model_load_widget.value + "を読み込みました。evaluateボタンを1回実行してください。")
model_load_button.on_click(load_model)
    
def save_model(c):
    path = "./model/"
    if not os.path.exists(path):
        subprocess.call(['mkdir', '-p', path])
    torch.save(model.state_dict(), path + model_save_name_widget.value)
    write_log("学習結果を" + path + model_save_name_widget.value + "に保存しました。")
model_save_button.on_click(save_model)

### ライブにデータセット作成と学習を実行

In [ ]:
import threading
import time
from utils import preprocess
import torch.nn.functional as F

state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
prediction_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)

def live(state_widget, model, camera, prediction_widget):
    global dataset
    while state_widget.value == 'live':
        image = camera.value
        preprocessed = preprocess(image)
        output = model(preprocessed).detach().cpu().numpy().flatten()
        category_index = dataset.categories.index(category_widget.value)
        x = output[2 * category_index]
        y = output[2 * category_index + 1]
        
        x = int(camera.width * (x / 2.0 + 0.5))
        y = int(camera.height * (y / 2.0 + 0.5))
        
        prediction = image.copy()
        prediction = cv2.circle(prediction, (x, y), 8, (255, 0, 0), 3)
        prediction_widget.value = bgr8_to_jpeg(prediction)
            
def start_live(change):
    if change['new'] == 'live':
        execute_thread = threading.Thread(target=live, args=(state_widget, model, camera, prediction_widget))
        execute_thread.start()

state_widget.observe(start_live, names='value')

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])

display(live_execution_widget)

In [ ]:
BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

epochs_widget = ipywidgets.IntText(description='epochs', value=1)
eval_button = ipywidgets.Button(description='evaluate')
train_button = ipywidgets.Button(description='train')
loss_widget = ipywidgets.FloatText(description='loss')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')

def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget
    
    try:
        if is_training:
            write_log("学習を開始します(50枚で10秒/1epoch, 100枚で20秒/1epoch,150枚で30秒/1epoch)")
        else:
            write_log("評価を開始します。")
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        state_widget.value = 'stop'
        train_button.disabled = True
        eval_button.disabled = True
        time.sleep(1)
        if is_training:
            model = model.train()
        else:
            model = model.eval()
        total_start_time = time.time()
        epoch_count = 1
        while epochs_widget.value > 0:
            start_time = time.time()
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            for images, category_idx, xy in iter(train_loader):
                # send data to device
                images = images.to(device)
                xy = xy.to(device)
                
                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()

                # execute model to get outputs
                outputs = model(images)

                # compute MSE loss over x, y coordinates for associated categories
                loss = 0.0
                for batch_idx, cat_idx in enumerate(list(category_idx.flatten())):
                    loss += torch.mean((outputs[batch_idx][2 * cat_idx:2 * cat_idx+2] - xy[batch_idx])**2)
                loss /= len(category_idx)
                
                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()
                # increment progress
                count = len(category_idx.flatten())
                i += count
                sum_loss += float(loss)
                progress_widget.value = i / len(dataset)
                loss_widget.value = sum_loss / i
            if is_training:
                end_time = time.time() - start_time
                write_log(str(epoch_count)+"epoch目が完了(処理時間:" + str(round(end_time,2)) + "秒)")
                epochs_widget.value = epochs_widget.value - 1
                epoch_count += 1
            else:
                break
        total_end_time = time.time() - total_start_time
        if is_training:
            write_log("学習を完了しました:(処理時間合計:" + str(round(total_end_time,2)) + "秒)")
        else:
            write_log("評価を完了しました。")
    except:
        write_log("Error")
        pass
    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False
    state_widget.value = 'live'
    
train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))
    
train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    ipywidgets.HBox([train_button, eval_button])
])

display(train_eval_widget)

### ファイルの読込保存

### 全部まとめて表示

以下のウィジェットは，複数クラスのx, yデータセットのラベル付けに使用できます． 1つの画像に対して，各クラスのインスタンスを1つだけラベリングすることをサポートしていますが（例：犬は1つだけ），1つの画像に対して複数のクラス（例：犬，猫，馬）をラベリングすることも可能です．

左上の画像をクリックすると、``category``の画像が、クリックした場所の``dataset``に保存されます。

| Widget | Description |
|--------|-------------|
| dataset | datasetを選択する |
| category | categoryを選択する |
| epochs | 転移学習のためのエポック数の数値を設定する |
| train | エポック数で指定された階数だけ選択されたデータセットを学習する  |
| evaluate | 選択したデータセットの1エポックあたりの精度を評価する |
| model path | モデルのパス名を指定 |
| load | model pathで指定した、学習済みモデルを読込 |
| save | model pathで指定した、学習済みモデルを保存 |
| stop | ライブデモを停止する |
| live | ライブデモを起動する |

In [ ]:
all_widget = ipywidgets.VBox([
    ipywidgets.HBox([data_collection_widget, live_execution_widget]), 
    train_eval_widget,
    ipywidgets.HBox([model_load_widget,model_load_time_widget,model_load_button]),
    ipywidgets.HBox([model_save_name_widget,model_save_button]),
    process_widget,
])

display(all_widget)

### カメラの終了処理(必須)

次のNotebookに進む際に必ずカメラの終了処理を実行してください。<br>
エラーがでますが、エラーがでた場合でも、処理が正しく実行されています。

In [ ]:
import time

camera.running = False
camera.cap.release()